In [25]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# CLEAN

In [24]:
# Reads in json data as a pandas DataFrame

tweets = pd.read_csv('/Users/veronicagiannotta/DSI/Projects/project-4/Optimizing-Evac-Routes/data/AT_historical/USA_142tweets.csv')

# Uncomment below to display the full text in each cell
# pd.set_option('display.max_colwidth', -1)
#[['coordinates', 'created_at', 'full_text', 'geo', 'id', 'place', 'user']]

tweets = tweets[['text','coordinates.coordinates',
       'created_at', 
       'place.bounding_box.coordinates',
       'place.country', 'place.country_code',
       'place.full_name', 'user.created_at',
       'user.location'
       ]]

tweets.shape

(143, 9)

In [23]:
tweets.isnull().sum()

text                               0
coordinates.coordinates           46
created_at                         0
place.bounding_box.coordinates     0
place.country                      0
place.country_code                 0
place.full_name                    0
user.created_at                    0
user.location                      6
dtype: int64

In [30]:
pd.set_option('display.max_colwidth', -1)
for i in tweets:
    print(tweets.text)
    print('___________________________________________________________________')
    print('   ')

0      Road construction, left lane closed in #Albuquerque on Tijeras Ave EB west of 3rd St #traffic https://t.co/roJyGryxFq                            
1      Road construction. right lanes closed in #Pima on I-10 EB at Ruthrauff Rd #traffic https://t.co/bc4TAizExm                                       
2      Road construction, shoulder closed in #ElPaso on I 10 Both EB/WB from Executive Ctr Blvd to Sunland Park Dr #traffic https://t.co/nZ2qPHc0XJ     
3      Ughhh at the dentist for a cleaning and the sidewalks are closed. HOW DO YOU CLOSE A SIDEWALK. Like...am I supposed to walk in the road lol      
4      Road constructions. two right lanes closed in #Pima on I-10 EB at Ruthrauff Rd #traffic https://t.co/bc4TAizExm                                  
5      All eastbound lanes are closed due to snow and ice on the road. in #Valyermo on Angeles Crest Hwy EB between CA-39 and Big Pines Hwy             
6      State Rd 44 is closed at Sugar Creek Rd near the Johnson County/Shelby Coun

In [133]:
train = pd.read_csv('/Users/veronicagiannotta/Desktop/tweets_train.csv')
test = pd.read_csv('/Users/veronicagiannotta/Desktop/tweets_test.csv')

In [134]:
train.drop('Unnamed: 0', axis=1, inplace=True)
train.head()

,text,intersection,coordinates.coordinates,created_at,place.bounding_box.coordinates,place.country,place.country_code,place.full_name,user.created_at,user.location
0,"Closed due to road construction on 288 S Fwy NB at The S Lp, stopped traffic back to Reed Rd. #traffic https://t.co/vuUN2yElDh",288 S Fwy NB at The S Lp,"[-95.38122, 29.68057]",Mon Jan 14 03:45:11 +0000 2019,"[[[-95.823268, 29.522325], [-95.069705, 29.522325], [-95.069705, 30.1546646], [-95.823268, 30.1546646]]]",United States,US,"Houston, TX",Wed Feb 09 21:09:17 +0000 2011,"Houston, TX"
1,"Road construction. two left lanes closed in #FortWorth on 35W SB at Allen Ave, stop and go traffic back to Rosedale St, delay of 1 min",35W SB at Allen Ave,"[-97.3204, 32.72612]",Mon Jan 14 03:23:02 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569477], [-97.033542, 32.990456], [-97.538285, 32.990456]]]",United States,US,"Fort Worth, TX",Thu May 21 21:15:33 +0000 2009,Dallas/Fort Worth
2,"Closed due to road construction in #FortWorth on 35W SB at Pharr, stop and go traffic back to Northside Dr… https://t.co/AyfxrFRbqU",35W SB at Pharr,"[-97.31861, 32.7677]",Mon Jan 14 10:33:20 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569477], [-97.033542, 32.990456], [-97.538285, 32.990456]]]",United States,US,"Fort Worth, TX",Thu May 21 21:15:33 +0000 2009,Dallas/Fort Worth
3,Closed due to road construction in #FortWorth on 35W SB at Pharr #DFWTraffic https://t.co/4NzBppXRvr,35W SB at Pharr,"[-97.31861, 32.7677]",Mon Jan 14 10:32:16 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569477], [-97.033542, 32.990456], [-97.538285, 32.990456]]]",United States,US,"Fort Worth, TX",Thu May 21 21:15:33 +0000 2009,Dallas/Fort Worth
4,"Road construction. three left lanes closed in #FortWorth on 35W SB between Allen Ave and Seminary Dr, stop and go... https://t.co/4NzBppXRvr",35W SB between Allen Ave and Seminary Dr,"[-97.3204, 32.72612]",Mon Jan 14 04:01:12 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569477], [-97.033542, 32.990456], [-97.538285, 32.990456]]]",United States,US,"Fort Worth, TX",Thu May 21 21:15:33 +0000 2009,Dallas/Fort Worth


In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 10 columns):
text                              26 non-null object
intersection                      26 non-null object
coordinates.coordinates           23 non-null object
created_at                        26 non-null object
place.bounding_box.coordinates    26 non-null object
place.country                     26 non-null object
place.country_code                26 non-null object
place.full_name                   26 non-null object
user.created_at                   26 non-null object
user.location                     26 non-null object
dtypes: object(10)
memory usage: 11.3+ KB


In [136]:
train.isnull().sum()

text                              118
intersection                      118
coordinates.coordinates           121
created_at                        118
place.bounding_box.coordinates    118
place.country                     118
place.country_code                118
place.full_name                   118
user.created_at                   118
user.location                     118
dtype: int64

In [137]:
train.dropna(axis=0, inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 0 to 25
Data columns (total 10 columns):
text                              23 non-null object
intersection                      23 non-null object
coordinates.coordinates           23 non-null object
created_at                        23 non-null object
place.bounding_box.coordinates    23 non-null object
place.country                     23 non-null object
place.country_code                23 non-null object
place.full_name                   23 non-null object
user.created_at                   23 non-null object
user.location                     23 non-null object
dtypes: object(10)
memory usage: 2.0+ KB


In [138]:
test.drop('Unnamed: 0', axis=1, inplace=True)
test.head()

,text,coordinates.coordinates,created_at,place.bounding_box.coordinates,place.country,place.country_code,place.full_name,user.created_at,user.location
0,Ughhh at the dentist for a cleaning and the sidewalks are closed. HOW DO YOU CLOSE A SIDEWALK. Like...am I supposed to walk in the road lol,NaN,Mon Jan 14 14:08:05 +0000 2019,"[[[-86.044756, 41.426657], [-85.9629436, 41.426657], [-85.9629436, 41.464375], [-86.044756, 41.464375]]]",United States,US,"Nappanee, IN",Thu Jul 30 19:27:12 +0000 2009,"New Paris, IN"
1,"Due to road conditions, schools and offices, including Central Office, will be closed for 12-month employees today, Monday, January 14.",NaN,Mon Jan 14 12:51:37 +0000 2019,"[[[-75.689007, 38.275414], [-75.6022644, 38.275414], [-75.6022644, 38.342711], [-75.689007, 38.342711]]]",United States,US,"Fruitland, MD",Tue Jul 26 18:57:36 +0000 2016,"Maryland, USA"
2,"For Monday, January 14, all RCPS schools wii be closed due to inclement weather and some hazardous road, parking lo… https://t.co/EXNvu2c2ST",NaN,Sun Jan 13 21:30:54 +0000 2019,"[[[-80.595589, 37.0900216], [-80.513486, 37.0900216], [-80.513486, 37.146017], [-80.595589, 37.146017]]]",United States,US,"Radford, VA",Sat Mar 15 23:41:20 +0000 2014,"Radford, VA"
3,"From ACPS... Due to anticipated road conditions tomorrow, Monday 1/14, schools will be closed. Stay safe, Mustangs… https://t.co/mEHzhndync",NaN,Sun Jan 13 21:08:40 +0000 2019,"[[[-83.67529, 36.540739], [-75.16644, 36.540739], [-75.16644, 39.466012], [-83.67529, 39.466012]]]",United States,US,"Virginia, USA",Wed Oct 29 14:34:42 +0000 2014,"Charlottesville, VA"
4,"Tolton Catholic will remain closed tomorrow, January 14, due to concerns about road conditions. Please watch commu… https://t.co/ju7ZMZnsIp",NaN,Sun Jan 13 19:24:49 +0000 2019,"[[[-95.774704, 35.995476], [-89.098843, 35.995476], [-89.098843, 40.613641], [-95.774704, 40.613641]]]",United States,US,"Missouri, USA",Fri Apr 08 03:15:39 +0000 2011,"Columbia, MO"


In [165]:
test.dropna(axis=0, inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 9 to 116
Data columns (total 9 columns):
text                              74 non-null object
coordinates.coordinates           74 non-null object
created_at                        74 non-null object
place.bounding_box.coordinates    74 non-null object
place.country                     74 non-null object
place.country_code                74 non-null object
place.full_name                   74 non-null object
user.created_at                   74 non-null object
user.location                     74 non-null object
dtypes: object(9)
memory usage: 5.8+ KB


# MODEL

In [178]:
X_train = train['text']
y_train = train['intersection']

X_test = test['text']

In [179]:
cv = CountVectorizer()
tf = TfidfVectorizer()

lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
nb = MultinomialNB()
bg = BaggingClassifier()

In [180]:
X_text = tf.fit_transform(X_train)

# Create a DataFrame to examine
# Note that pandas has a `.SparseDataFrame` method that will keep your data sparse (which will save RAM) 
# while allowing you to use pandas functions
text = pd.DataFrame(X_text.toarray(), columns=tf.get_feature_names())
text.head()

,10,119,275,288,35w,39,3rd,40,4nzbppxrvr,6th,...,tollway,traffic,two,us,valyermo,vly,vuun2yeldh,water,wb,west
0,0.0,0.0,0.0,0.253535,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.307528,0.00000,0.0,0.0,0.0,0.253535,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.172756,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.155962,0.21981,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.192993,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.174232,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.271949,0.0,0.0,0.0,0.346020,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.203197,0.0,0.0,0.0,0.258542,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [181]:
lr.fit(text, y_train)

lr.score(text, y_train)

0.7391304347826086

In [182]:
X_test_text = tf.transform(X_test)

# Create a DataFrame to examine
# Note that pandas has a `.SparseDataFrame` method that will keep your data sparse (which will save RAM) 
# while allowing you to use pandas functions
test_text = pd.DataFrame(X_test_text.toarray(), columns=tf.get_feature_names())
test_text.head()

,10,119,275,288,35w,39,3rd,40,4nzbppxrvr,6th,...,tollway,traffic,two,us,valyermo,vly,vuun2yeldh,water,wb,west
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.667301,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.230513,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.230513,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.287287,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.316892,0.174234,0.0,0.000000,0.000000,0.0,0.287287,0.0,0.0,0.0


In [185]:
# This cell doesn't run without an error, perhaps there is an issue with the text encoding? 
# the .astype(str) was an attempt to get things working, but it didn't seem to solve the problem...
preds = lr.predict(test_text)

lr.score(y_train.astype(str), preds.astype(str))

ValueError: could not convert string to float: '288 S Fwy NB at The S Lp'